In [1]:
import os
import collections
import SimpleITK as sitk
import numpy 
import six
import radiomics
from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm, ngtdm, gldm

In [2]:
from imgtools.io import read_dicom_series
from readii.loaders import loadSegmentation
from readii.image_processing import flattenImage, alignImages, displayImageSlice, displayCTSegOverlay, getCroppedImages, getROIVoxelLabel, getROICenterCoords

# Load image data

In [ ]:
# RADCRE-2012
ctDirPath = "/Users/katyscott/Documents/HNC Project/data/error_images/RADCURE/RADCURE-2012/12-26-2001-NA-Research HNC Planning CT-39063/3.000000-Helical Axial-26282"
segFilePath = "/Users/katyscott/Documents/HNC Project/data/error_images/RADCURE/RADCURE-2012/12-26-2001-NA-Research HNC Planning CT-39063/1.000000-NA-03155/1-1.dcm"

In [ ]:
# RADCURE-3716
ctDirPath = "/Users/katyscott/Documents/HNC Project/data/error_images/RADCURE/RADCURE-3716/20091006-Research HNC Planning CT-1.3.6.1.4.1.14519.5.2.1.159479891309173287983489538813569798675/3-Neck 2.0 CE-1.3.6.1.4.1.14519.5.2.1.303618333208483957981299429120448318183"
segFilePath = "/Users/katyscott/Documents/HNC Project/data/error_images/RADCURE/RADCURE-3716/20091006-Research HNC Planning CT-1.3.6.1.4.1.14519.5.2.1.159479891309173287983489538813569798675/1-UnknownSeriesDescription-1.3.6.1.4.1.14519.5.2.1.53689980185072466582338154816320570464/1.3.6.1.4.1.14519.5.2.1.196814539827040265136159678638398372874.dcm"

In [ ]:
# RADCURE-2818
segFilePath = "/Users/katyscott/Documents/HNC Project/data/error_images/RADCURE/RADCURE-2818/20050706-Research HNC Planning CT-1.3.6.1.4.1.14519.5.2.1.32217992229678458029795766308716011483/1-UnknownSeriesDescription-1.3.6.1.4.1.14519.5.2.1.140290746105280466409401562789091483595/1.3.6.1.4.1.14519.5.2.1.242324221082044849247326651271356162325.dcm"
ctDirPath = "/Users/katyscott/Documents/HNC Project/data/error_images/RADCURE/RADCURE-2818/20050706-Research HNC Planning CT-1.3.6.1.4.1.14519.5.2.1.32217992229678458029795766308716011483/2-Neck 2.0 CE-1.3.6.1.4.1.14519.5.2.1.309190676350470239537900264663032220893"

In [3]:
#RADCURE-2840
ctDirPath = "/Users/katyscott/Documents/HNC Project/data/error_images/RADCURE/RADCURE-2840/20050816-Research HNC Planning CT-1.3.6.1.4.1.14519.5.2.1.287491432052519970379240880654397710367/2-Neck 2.0 CE-1.3.6.1.4.1.14519.5.2.1.55419138121179541338800071683548264399"
segFilePath = "/Users/katyscott/Documents/HNC Project/data/error_images/RADCURE/RADCURE-2840/20050816-Research HNC Planning CT-1.3.6.1.4.1.14519.5.2.1.287491432052519970379240880654397710367/1-UnknownSeriesDescription-1.3.6.1.4.1.14519.5.2.1.303274724757270970302406387115992451078/1.3.6.1.4.1.14519.5.2.1.75523572134527947195118599778613464026.dcm"

In [4]:
ctImage = read_dicom_series(ctDirPath)
segImages = loadSegmentation(segImagePath=segFilePath,
                             modality='RTSTRUCT',
                             baseImageDirPath=ctDirPath,
                             roiNames='GTVp.*')

labels: {'GTVp': 0}


In [5]:
roiLabel = list(segImages.keys())[0]
roiImage = segImages[roiLabel]

In [6]:
print("CT dimensions: ", ctImage.GetSize())
print("Segmentation dimensions: ", roiImage.GetSize())

CT dimensions:  (512, 512, 178)
Segmentation dimensions:  (512, 512, 178)


In [7]:
roiMask = alignImages(ctImage, roiImage)

In [8]:
segmentationLabel: int = getROIVoxelLabel(roiMask)

# Make Negative Control

In [11]:
from readii.negative_controls import *
import numpy as np

randomSeed = 10

In [ ]:
fullImgCenterCoords = getROICenterCoords(roiMask)
print(fullImgCenterCoords)

In [ ]:
negativeControlType = "randomized" 
negativeControlRegion = "full"

ctImage = applyNegativeControl(baseImage = ctImage, 
                               negativeControlType = negativeControlType, 
                               negativeControlRegion = negativeControlRegion, 
                               roiMask = roiMask, 
                               randomSeed = randomSeed)

# PyRadiomics Settings Setup

In [ ]:
settings = {}
settings['binWidth'] = 25
settings['resampledPixelSpacing'] = [1., 1., 1.]
settings['interpolator'] = 'sitkBSpline'

In [ ]:
# Resample if necessary
interpolator = settings.get('interpolator')
resampledPixelSpacing = settings.get('resampledPixelSpacing')
if interpolator is not None and resampledPixelSpacing is not None:
  ctImage, roiMask = imageoperations.resampleImage(ctImage, roiMask, **settings)

In [ ]:
segBoundingBox, correctedROIMask = imageoperations.checkMask(ctImage, roiMask, label = segmentationLabel)
    
# Update the ROI image if a correction was generated by checkMask
if correctedROIMask is not None:
    roiMask = correctedROIMask

# Crop the image and mask to a bounding box around the mask to reduce volume size to process
croppedCT, croppedROIMask = imageoperations.cropToTumorMask(ctImage, roiMask, segBoundingBox)

In [ ]:
croppedCT.GetSize()

In [ ]:
negativeControlType = "randomized_sampled_non_roi"

croppedCT = applyNegativeControl(negativeControlType, croppedCT, croppedROIMask, 
                               roiLabel, randomSeed)

# Calculate Shape Features

In [ ]:
shapeFeatures = shape.RadiomicsShape(croppedCT, croppedROIMask, **settings)

# Set the features to be calculated
# shapeFeatures.enableFeatureByName('MeshVolume', True)
shapeFeatures.enableAllFeatures()

In [ ]:
# Calculate the features and print(out result)
print('Calculating shape features...',)
result = shapeFeatures.execute()
print('done')

print('Calculated shape features: ')
for (key, val) in six.iteritems(result):
  print('  ', key, ':', val)

# Calculate GLCM Features

In [ ]:
radiomics.setVerbosity(10)

In [ ]:
glcmFeatures = glcm.RadiomicsGLCM(croppedCT, croppedROIMask, **settings)

# Set the features to be calculated
# glcmFeatures.enableFeatureByName('MCC', True)
glcmFeatures.enableAllFeatures()

In [ ]:
# Calculate the features and print(out result)
print('Calculating GLCM features...',)
result = glcmFeatures.execute()
print('done')

print('Calculated GLCM features: ')
for (key, val) in six.iteritems(result):
  print('  ', key, ':', val)

# Wavelet filtering

In [ ]:
radiomics.setVerbosity(10)

In [ ]:
waveletFeatures = {}

for decompositionImage, decompositionName, inputSettings in imageoperations.getWaveletImage(ctImage, roiMask):

    # if decompositionName == "wavelet-HLL":
    decompositionImage, croppedMask = imageoperations.cropToTumorMask(decompositionImage, roiMask, segBoundingBox)

    # waveletFirstOrderFeaturs = firstorder.RadiomicsFirstOrder(decompositionImage, croppedMask, **inputSettings)
    # waveletFirstOrderFeaturs.enableAllFeatures()

    # print('Calculate firstorder features with ', decompositionName)
    # waveletFeatures[decompositionName] = waveletFirstOrderFeaturs.execute()

    # waveletGLSZMFeatures = glszm.RadiomicsGLSZM(decompositionImage, croppedMask, **inputSettings)
    # waveletGLSZMFeatures.enableAllFeatures()

    # waveletNGTDMFeatures = ngtdm.RadiomicsNGTDM(decompositionImage, croppedMask, **inputSettings)
    # waveletNGTDMFeatures.enableAllFeatures()

    # waveletGLDMFeatures = gldm.RadiomicsGLDM(decompositionImage, croppedMask, **inputSettings)
    # waveletGLDMFeatures.enableAllFeatures()
    
    waveletGLCMFeatures = glcm.RadiomicsGLCM(decompositionImage, croppedMask, **inputSettings)
    waveletGLCMFeatures.disableAllFeatures()
    waveletGLCMFeatures.enableFeatureByName("MCC")
    # waveletGLCMFeatures.enableAllFeatures()

    print('Calculate features with ', decompositionName)
    waveletFeatures[decompositionName] = waveletGLCMFeatures.execute()

In [ ]:
waveletFeatures['wavelet-HLL']

# Using READII 

In [9]:
radiomics.setVerbosity(10)

In [10]:
from readii.feature_extraction import singleRadiomicFeatureExtraction

# pyradiomicsParamFilePath = "/Users/katyscott/Documents/readii/src/readii/data/default_pyradiomics.yaml"
pyradiomicsParamFilePath = "/Users/katyscott/Documents/HNC Project/scripts/hnc_readii-2-roqc/config/no_wavelet_pyradiomics.yaml"
# pyradiomicsParamFilePath = "/Users/katyscott/Documents/HNC Project/scripts/hnc_readii-2-roqc/config/original_and_wavelet_pyradiomics.yaml"
radiomicFeatures = singleRadiomicFeatureExtraction(ctImage, roiMask, pyradiomicsParamFilePath, randomSeed=10)

Checking mask with label 1
Calculating bounding box
Checking minimum number of dimensions requirements (2)
Cropping to size [56 71 22]
Loading parameter file /Users/katyscott/Documents/HNC Project/scripts/hnc_readii-2-roqc/config/no_wavelet_pyradiomics.yaml
Parameters parsed, input is valid.
Applying settings
Enabled image types: {'Original': {}, 'Square': {}, 'SquareRoot': {}, 'Logarithm': {}, 'Exponential': {}, 'Gradient': {}}
Enabled features: {'shape': None, 'firstorder': None, 'glcm': None, 'glrlm': None, 'glszm': None, 'gldm': None, 'ngtdm': None}
Settings: {'binWidth': 25, 'resampledPixelSpacing': [1.0, 1.0, 1.0]}
Calculating features with label: 1
Enabled images types: {'Original': {}, 'Square': {}, 'SquareRoot': {}, 'Logarithm': {}, 'Exponential': {}, 'Gradient': {}}
Enabled features: {'shape': None, 'firstorder': None, 'glcm': None, 'glrlm': None, 'glszm': None, 'gldm': None, 'ngtdm': None}
Current settings: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': Fal

Starting radiomic feature extraction...


Shape feature class initialized
Calculating features
Creating image type iterator
Adding image type "Original" with custom settings: {}
Adding image type "Square" with custom settings: {}
Adding image type "SquareRoot" with custom settings: {}
Adding image type "Logarithm" with custom settings: {}
Adding image type "Exponential" with custom settings: {}
Adding image type "Gradient" with custom settings: {}
Extracting features
Yielding original image
Calculating features for original image
Cropping to size [51 65 44]
Computing firstorder
Initializing feature class
Discretizing gray levels inside ROI
Calculated 1379 bins for bin width 25 with edges: [-3625 -3600 -3575 ... 30800 30825 30850])
First order feature class initialized
Calculating features
Computing glcm
Initializing feature class
Discretizing gray levels inside ROI
Calculated 1379 bins for bin width 25 with edges: [-3625 -3600 -3575 ... 30800 30825 30850])
Calculating GLCM matrix in C
Process calculated matrix
Create symmetric

In [ ]:
import pandas as pd
pd.DataFrame.from_dict(radiomicFeatures, orient="index").transpose()

In [ ]:
for (key, val) in six.iteritems(radiomicFeatures):
  print('  ', key, ':', val)

# Use singleRadiomicFeatureExtraction function 

In [16]:
from readii.feature_extraction import singleRadiomicFeatureExtraction 
from func_timeout import func_timeout, FunctionTimedOut

try:
    pyradiomicsParamFilePath = "/Users/katyscott/Documents/readii/src/readii/data/default_pyradiomics.yaml"
    radiomicFeatures = func_timeout(timeout = 1800, func = singleRadiomicFeatureExtraction, args = (ctImage, roiMask, pyradiomicsParamFilePath, "randomized_roi", 10))
except FunctionTimedOut:
    print("Default radiomics features took too long to run. Running without wavelet filter.")
    pyradiomicsParamFilePath = "/Users/katyscott/Documents/HNC Project/scripts/hnc_readii-2-roqc/config/no_wavelet_pyradiomics.yaml"
    radiomicFeatures = singleRadiomicFeatureExtraction(ctImage, roiMask, pyradiomicsParamFilePath, None, 10)
except Exception as e:
    print(e)



Checking mask with label 1
Calculating bounding box
Checking minimum number of dimensions requirements (2)


Generating  randomized_roi negative control for CT.


Cropping to size [56 71 22]
Loading parameter file /Users/katyscott/Documents/readii/src/readii/data/default_pyradiomics.yaml
Parameters parsed, input is valid.
Applying settings
Enabled image types: {'Original': {}, 'Wavelet': {}, 'Square': {}, 'SquareRoot': {}, 'Logarithm': {}, 'Exponential': {}, 'Gradient': {}}
Enabled features: {'shape': None, 'firstorder': None, 'glcm': None, 'glrlm': None, 'glszm': None, 'gldm': None, 'ngtdm': None}
Settings: {'binWidth': 25, 'resampledPixelSpacing': [1.0, 1.0, 1.0]}
Calculating features with label: 1
Enabled images types: {'Original': {}, 'Wavelet': {}, 'Square': {}, 'SquareRoot': {}, 'Logarithm': {}, 'Exponential': {}, 'Gradient': {}}
Enabled features: {'shape': None, 'firstorder': None, 'glcm': None, 'glrlm': None, 'glszm': None, 'gldm': None, 'ngtdm': None}
Current settings: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': [1.0, 1.0, 1.0], 'interpolat

Starting radiomic feature extraction...


First order feature class initialized
Calculating features
Computing glcm
Initializing feature class
Discretizing gray levels inside ROI
Calculated 1740 bins for bin width 25 with edges: [-10700 -10675 -10650 ...  32750  32775  32800])
Calculating GLCM matrix in C
Process calculated matrix
Create symmetrical matrix
No empty angles
Calculating GLCM coefficients
GLCM feature class initialized, calculated GLCM with shape (1, 1546, 1546, 13)
Calculating features
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
Computing glrlm
Initializing feature class
Discretizing gray levels inside ROI
Calculated 1740 bins for bin width 25 with edges: [-10700 -10675 -10650 ...  32750  32775  32800])
Calculating GLRLM matrix in C
Process calculated matrix
No empty angles
Calculating GLRLM coefficients
GLRLM feature class initialized, calculated GLRLM with shape (1, 1546, 2, 13)
Calculating features
Computing glszm
Initializing feature class
Discretizing gray le

Default radiomics features took too long to run. Running without wavelet filter.


Checking minimum number of dimensions requirements (2)
Cropping to size [56 71 22]
Loading parameter file /Users/katyscott/Documents/HNC Project/scripts/hnc_readii-2-roqc/config/no_wavelet_pyradiomics.yaml
Parameters parsed, input is valid.
Applying settings
Enabled image types: {'Original': {}, 'Square': {}, 'SquareRoot': {}, 'Logarithm': {}, 'Exponential': {}, 'Gradient': {}}
Enabled features: {'shape': None, 'firstorder': None, 'glcm': None, 'glrlm': None, 'glszm': None, 'gldm': None, 'ngtdm': None}
Settings: {'binWidth': 25, 'resampledPixelSpacing': [1.0, 1.0, 1.0]}
Calculating features with label: 1
Enabled images types: {'Original': {}, 'Square': {}, 'SquareRoot': {}, 'Logarithm': {}, 'Exponential': {}, 'Gradient': {}}
Enabled features: {'shape': None, 'firstorder': None, 'glcm': None, 'glrlm': None, 'glszm': None, 'gldm': None, 'ngtdm': None}
Current settings: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 're

Starting radiomic feature extraction...


Shape feature class initialized
Calculating features
Creating image type iterator
Adding image type "Original" with custom settings: {}
Adding image type "Square" with custom settings: {}
Adding image type "SquareRoot" with custom settings: {}
Adding image type "Logarithm" with custom settings: {}
Adding image type "Exponential" with custom settings: {}
Adding image type "Gradient" with custom settings: {}
Extracting features
Yielding original image
Calculating features for original image
Cropping to size [51 65 44]
Computing firstorder
Initializing feature class
Discretizing gray levels inside ROI
Calculated 1379 bins for bin width 25 with edges: [-3625 -3600 -3575 ... 30800 30825 30850])
First order feature class initialized
Calculating features
Computing glcm
Initializing feature class
Discretizing gray levels inside ROI
Calculated 1379 bins for bin width 25 with edges: [-3625 -3600 -3575 ... 30800 30825 30850])
Calculating GLCM matrix in C
Process calculated matrix
Create symmetric

In [17]:
import pandas as pd
pd.DataFrame.from_dict(radiomicFeatures, orient="index").transpose()

,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,...,gradient_gldm_LargeDependenceLowGrayLevelEmphasis,gradient_gldm_LowGrayLevelEmphasis,gradient_gldm_SmallDependenceEmphasis,gradient_gldm_SmallDependenceHighGrayLevelEmphasis,gradient_gldm_SmallDependenceLowGrayLevelEmphasis,gradient_ngtdm_Busyness,gradient_ngtdm_Coarseness,gradient_ngtdm_Complexity,gradient_ngtdm_Contrast,gradient_ngtdm_Strength
0,v3.0.1a3,1.26.4,2.3.1,1.5.0,3.9.18,"{'minimumROIDimensions': 2, 'minimumROISize': ...","{'Original': {}, 'Square': {}, 'SquareRoot': {...",78e99f434f61bce26aaff1d2221b675ac1efd4c3,3D,"(0.919, 0.919, 2.0)",...,138.03149063522696,0.46473757325088433,0.11277028674550117,1026.9517314366883,0.0074465670195857665,1.0214470507517626,0.0004959373676274749,138342.4747045337,0.06420642868525576,834.8086224396645


In [21]:
from radiomics import featureextractor
import yaml

pyradiomicsParamFilePath = "/Users/katyscott/Documents/readii/src/readii/data/default_pyradiomics.yaml"

with open(pyradiomicsParamFilePath, 'r') as file:
    data = yaml.safe_load(file)

# imageTypes = data.pop('imageType')
# del imageTypes['Wavelet']
# data['imageType'] = imageTypes

# featureExtractor = featureextractor.RadiomicsFeatureExtractor(data)


In [22]:
del data['imageType']['Wavelet']

In [23]:
data

{'imageType': {'Original': {},
  'Square': {},
  'SquareRoot': {},
  'Logarithm': {},
  'Exponential': {},
  'Gradient': {}},
 'featureClass': {'shape': None,
  'firstorder': None,
  'glcm': None,
  'glrlm': None,
  'glszm': None,
  'gldm': None,
  'ngtdm': None},
 'setting': {'binWidth': 25, 'resampledPixelSpacing': [1.0, 1.0, 1.0]}}

In [20]:
idFeatureVector = featureExtractor.execute(ctImage, roiMask, label=segmentationLabel)

Calculating features with label: 1
Enabled images types: {'Original': {}, 'Square': {}, 'SquareRoot': {}, 'Logarithm': {}, 'Exponential': {}, 'Gradient': {}}
Enabled features: {'shape': None, 'firstorder': None, 'glcm': None, 'glrlm': None, 'glszm': None, 'gldm': None, 'ngtdm': None}
Current settings: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': [1.0, 1.0, 1.0], 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 25}
Loading image and mask
Force casting mask to UInt32 to ensure correct datatype.
Resampling image and mask
Where resampled spacing is set to 0, set it to the original spacing (mask)
Checking ROI validity
Checking if label 1 is persent in the mask
Comparing physical space of bounding box to physical space of image
ROI bounds (image coor